**Pharmacy Inventory Manager**

Core Features
*   Add a new drug (name, quantity, price, expiry date)
*   Update price
*   Update stock quantity
*   Sell medicine
*   Remove medicine
*   Check expiration date
*   Display inventory
*   Calculate total inventory value
*   Save and load data from a CSV file (so records persist)
*   Interactive menu interface for real-time use



In [ ]:
import json
from datetime import datetime, timedelta
import os

class Medicine:
    def __init__(self, name, quantity, price, expiry_date):
        self.name = name
        self.quantity = quantity
        self.price = price
        # Store expiry date as a datetime object
        self.expiry_date = datetime.strptime(expiry_date, "%Y-%m-%d")

    def to_dict(self):
        return {
            "name": self.name,
            "quantity": self.quantity,
            "price": self.price,
            "expiry_date": self.expiry_date.strftime("%Y-%m-%d")
        }

    @classmethod
    def from_dict(cls, data):
        return cls(data["name"], data["quantity"], data["price"], data["expiry_date"])


    def __str__(self):
        expiry_str = self.expiry_date.strftime("%Y-%m-%d")
        return f"{self.name} - {self.quantity} units - ₦{self.price:.2f}/unit - Exp: {expiry_str}"

In [ ]:
# Example of a medicine entry

m1= Medicine('paracetamol', 20, 30, '2027-9-15')
print(m1)

paracetamol - 20 units - ₦30.00/unit - Exp: 2027-09-15


In [ ]:
class Inventory:
    FILE_PATH = "inventory_data.json"

    def __init__(self):
        self.medicines = {}
        self.load_inventory()

    def save_inventory(self):
        data = {name: med.to_dict() for name, med in self.medicines.items()}
        with open(self.FILE_PATH, "w") as file:
            json.dump(data, file, indent=4)

    def load_inventory(self):
        if os.path.exists(self.FILE_PATH):
            with open(self.FILE_PATH, "r") as file:
                data = json.load(file)
                self.medicines = {name: Medicine.from_dict(info) for name, info in data.items()}
            print("Inventory loaded successfully.\n")
        else:
            print("No existing inventory found. Starting fresh.\n")

    def add_medicine(self, name, quantity, price, expiry_date):
        """Add new medicine or merge quantity if it exists"""
        if name in self.medicines:
            self.medicines[name].quantity += quantity
            print(f"Updated {name} quantity. New total: {self.medicines[name].quantity}")
        else:
            self.medicines[name] = Medicine(name, quantity, price, expiry_date)
            print(f"Added {name} to inventory.")

    def update_price(self, name, new_price):
        """Update price of an existing medicine"""
        if name in self.medicines:
            self.medicines[name].price = new_price
            print(f"Updated {name} price to ₦{new_price:.2f}")
        else:
            print("Medicine not found in inventory.")

    def update_quantity(self, name, quantity):
        """Update the quantity of a medicine"""
        if name in self.medicines:
            self.medicines[name].quantity += quantity
            print(f"Quantity for {name} updated. New total: {self.medicines[name].quantity}")
        else:
            print("Medicine not found in inventory.")

    def sell_medicine(self, name, quantity):
        """Sell medicine and reduce stock"""
        if name in self.medicines:
            med = self.medicines[name]
            if med.quantity >= quantity:
                med.quantity -= quantity
                print(f"Sold {quantity} units of {name}. Remaining: {med.quantity}")
            else:
                print(f"Not enough stock for {name}. Only {med.quantity} left.")
        else:
            print("Medicine not found in inventory.")

    def remove_medicine(self, name):
        """Remove medicine entry from inventory"""
        if name in self.medicines:
            del self.medicines[name]
            print(f"Removed {name} from inventory.")
        else:
            print("Medicine not found in inventory.")

    def check_expiry(self):
        """Notify if any medicine will expire within 3 months"""
        print("\nChecking for upcoming expiries...")
        today = datetime.now()
        three_months_later = today + timedelta(days=90)
        found = False

        for med in self.medicines.values():
            if today <= med.expiry_date <= three_months_later:
                found = True
                days_left = (med.expiry_date - today).days
                print(f"{med.name} expires in {days_left} days ({med.expiry_date.strftime('%Y-%m-%d')})")

        if not found:
            print("No medicines expiring within the next 3 months.")

    def display_inventory(self):
        """Show all medicines in inventory"""
        if not self.medicines:
            print("Inventory is empty.")
        else:
            print("\nCurrent Inventory:")
            for med in self.medicines.values():
                print(med)

In [ ]:
# ---- Example Usage ----
pharmacy = Inventory()

# adding a new medicine
pharmacy.add_medicine("Paracetamol", 50, 25, "2026-12-01")
pharmacy.add_medicine("Ibuprofen", 30, 50, "2027-11-10")
print('--' * 30)

# updating the quantity of a medicine
pharmacy.add_medicine("Paracetamol", 20, 25, "2026-12-01")
print('--' * 30)

# selling a medicine
pharmacy.sell_medicine("Ibuprofen", 5)
print('--' * 30)

# updating the price of a medicine
pharmacy.update_price("Paracetamol", 30)
print('--' * 30)

# displaying the inventory
pharmacy.display_inventory()
print('--' * 30)

# checking for expired drug
pharmacy.check_expiry()
print('--' * 30)

# removing a medicine entry
pharmacy.remove_medicine("Ibuprofen")
print('--' * 30)

# displaying the Inventory
pharmacy.display_inventory()

No existing inventory found. Starting fresh.

Added Paracetamol to inventory.
Added Ibuprofen to inventory.
------------------------------------------------------------
Updated Paracetamol quantity. New total: 70
------------------------------------------------------------
Sold 5 units of Ibuprofen. Remaining: 25
------------------------------------------------------------
Updated Paracetamol price to ₦30.00
------------------------------------------------------------

Current Inventory:
Paracetamol - 70 units - ₦30.00/unit - Exp: 2026-12-01
Ibuprofen - 25 units - ₦50.00/unit - Exp: 2027-11-10
------------------------------------------------------------

Checking for upcoming expiries...
No medicines expiring within the next 3 months.
------------------------------------------------------------
Removed Ibuprofen from inventory.
------------------------------------------------------------

Current Inventory:
Paracetamol - 70 units - ₦30.00/unit - Exp: 2026-12-01


# Interactive menu interface for real-time use

In [ ]:
def menu():
    inventory = Inventory()

    while True:
        print("""
========= PHARMACY INVENTORY MANAGER =========
1. Add Medicine
2. Update Price
3. Update Quantity
4. Sell Medicine
5. Remove Medicine
6. View Inventory
7. Check Expiry
8. Exit
===============================================
""")
        choice = input("Enter your choice (1–8): ")

        if choice == "1":
            name = input("Medicine name: ").capitalize()
            quantity = int(input("Quantity: "))
            price = float(input("Price (₦): "))
            expiry_date = input("Expiry date (YYYY-MM-DD): ")
            inventory.add_medicine(name, quantity, price, expiry_date)

        elif choice == "2":
            name = input("Medicine name: ").capitalize()
            new_price = float(input("New price (₦): "))
            inventory.update_price(name, new_price)

        elif choice == "3":
            name = input("Medicine name: ").capitalize()
            quantity = int(input("Quantity to add: "))
            inventory.update_quantity(name, quantity)

        elif choice == "4":
            name = input("Medicine name: ").capitalize()
            quantity = int(input("Quantity to sell: "))
            inventory.sell_medicine(name, quantity)

        elif choice == "5":
            name = input("Medicine name: ").capitalize()
            inventory.remove_medicine(name)

        elif choice == "6":
            inventory.display_inventory()

        elif choice == "7":
            inventory.check_expiry()

        elif choice == "8":
            print("Exiting system. Have a productive day!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    menu()

No existing inventory found. Starting fresh.


========= PHARMACY INVENTORY MANAGER =========
1. Add Medicine
2. Update Price
3. Update Quantity
4. Sell Medicine
5. Remove Medicine
6. View Inventory
7. Check Expiry
8. Exit

Enter your choice (1–8): 6
Inventory is empty.

========= PHARMACY INVENTORY MANAGER =========
1. Add Medicine
2. Update Price
3. Update Quantity
4. Sell Medicine
5. Remove Medicine
6. View Inventory
7. Check Expiry
8. Exit

Enter your choice (1–8): 1
Medicine name: paracetamol
Quantity: 30
Price (₦): 45
Expiry date (YYYY-MM-DD): 2026-12-1
Added Paracetamol to inventory.

========= PHARMACY INVENTORY MANAGER =========
1. Add Medicine
2. Update Price
3. Update Quantity
4. Sell Medicine
5. Remove Medicine
6. View Inventory
7. Check Expiry
8. Exit

Enter your choice (1–8): 3
Medicine name: paracetamol
Quantity to add: 3
Quantity for Paracetamol updated. New total: 33

========= PHARMACY INVENTORY MANAGER =========
1. Add Medicine
2. Update Price
3. Update Quantity
4. S